<a href="https://colab.research.google.com/github/OnePassio/CE888/blob/master/lab_5_my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5
https://github.com/albanda/CE888/tree/master/lab5-recommender

1.Load Database "jester-data-1.csv"
Format:
- Ratings are real values ranging from -10.00 to +10.00 (the value "99" corresponds to "null" = "not rated").
- One row per user
- The first column gives the number of jokes rated by that user. The next 100 columns give the ratings for jokes 01 - 100.
- The sub-matrix including only columns {5, 7, 8, 13, 15, 16, 17, 18, 19, 20} is dense. Almost all users have rated those jokes (see discussion of "universal queries" in the above paper).

In [206]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df_data= pd.read_excel('https://raw.githubusercontent.com/OnePassio/CE888/master/lab5/jester-data-1.xls',
                       header=None)
df_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


2.Using the helper function provided in the helper_function notebook (or create your own), label 10% of the dataset cells as 99. This is your validation set. Keep the the actual values of the cells so you can use them later.

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(orig!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [208]:
# original data
data_original=df_data.values
print(data_original.shape)

#rated = np.where(data_original!=99)
#print(len(rated[0]), rated[1].shape)

(24983, 101)


In [209]:
#new data with 10% replace value 99
data_new, idx = replace(data_original, 0.1)
print(data_new.shape)
data_new

(24983, 101)


array([[ 74.  ,  -7.82,   8.79, ...,  99.  ,  99.  ,  99.  ],
       [100.  ,  99.  ,  -0.29, ...,   0.34,  -4.32,   1.07],
       [ 49.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       ...,
       [ 39.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 99.  ,   2.43,   2.67, ...,  99.  ,  99.  ,  99.  ]])

3.Use latent factor modeling to infer the hidden ratings of the users (they are labeled as "99" in the dataset) on the training set

In [0]:
#data without first column
data_new_2 = np.delete(data_new, 0, 1)
#print(data_new_2.shape)
#data_new

# Initialise as random values
n_latent_factors = 2
latent_user_preferences = np.random.random((data_new_2.shape[0], n_latent_factors))
latent_item_features = np.random.random((data_new_2.shape[1], n_latent_factors))



In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30,print_value=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = data_new_2[user_id][item_id]
                if abs(rating-99)>0.1: # rating is not 99.0  -->not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % print_value) == 0:
          print("iteration: {},mse: {}".format(iteration,mse))
        

In [212]:
sgd(30,2)

iteration: 0,mse: 26.192667036226595
iteration: 2,mse: 24.994686780369815
iteration: 4,mse: 24.56575328267493
iteration: 6,mse: 24.135690827638133
iteration: 8,mse: 23.69837445029411
iteration: 10,mse: 23.26277325143068
iteration: 12,mse: 22.8370205812805
iteration: 14,mse: 22.42522145340205
iteration: 16,mse: 22.02494992043252
iteration: 18,mse: 21.624978329430974
iteration: 20,mse: 21.204101736855222
iteration: 22,mse: 20.73412336778589
iteration: 24,mse: 20.192845849713034
iteration: 26,mse: 19.589109011598985
iteration: 28,mse: 18.980637730722577


 make predictions for the test dataset

In [213]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-1.9371669 , -3.0645159 , -3.20678077, ..., -1.37044334,
        -3.34553267, -1.47464686],
       [ 1.90745677,  2.04005195,  1.96622534, ...,  2.09769317,
         1.81423165,  2.41002395],
       [ 4.6434213 ,  4.80231001,  4.58672309, ...,  5.23198833,
         4.16832345,  6.02747678],
       ...,
       [ 0.07833635, -0.2038921 , -0.26987903, ...,  0.30636925,
        -0.3610632 ,  0.38091975],
       [ 0.26021721, -0.21401265, -0.33182387, ...,  0.66304927,
        -0.49792686,  0.81129004],
       [ 2.38674795,  2.04048072,  1.83601732, ...,  3.01686953,
         1.49459948,  3.51757416]])

In [0]:
values = [zip(data_new_2[i], predictions[i]) for i in range(predictions.shape[0])]
 
comparison_data = pd.DataFrame(values)
first_column=data_new[:, 0]

first_column=np.reshape(first_column,(first_column.shape[0],1))

comparison_data=np.concatenate((first_column, comparison_data),axis=1)
comparison_data=pd.DataFrame(comparison_data)
comparison_data.columns = df_data.columns



In [215]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,"(-7.82, -1.9371668996400855)","(8.79, -3.0645158979011784)","(-9.66, -3.2067807709151372)","(-8.16, -6.008009073828286)","(-7.52, -1.96208784546421)","(-8.5, -1.2780663774721168)","(-9.85, -2.9901729700054727)","(4.17, -2.5193988890421486)","(-8.98, -5.017353817374149)","(-4.76, -1.9851737816494084)","(-8.5, -1.2625548311893546)","(-6.75, -1.571904613561203)","(-7.18, -4.77969437072656)","(8.45, -1.7644079793683372)","(-7.18, -4.33956822047977)","(-7.52, -6.606704262473739)","(-7.43, -3.1475672429965433)","(-9.81, -3.433814062906405)","(-9.85, -2.6260997615616035)","(-9.85, -4.352921350661272)","(-9.37, -0.3784250635615159)","(1.5, -3.153410633010008)","(-4.37, -4.0944481689978645)","(-9.81, -6.9341520250888165)","(-8.5, -3.5169622568095438)","(1.12, -1.7353909001830061)","(7.82, 1.8431056624089386)","(2.86, -1.5352481941886968)","(99.0, 2.1708016342674017)","(-7.43, -5.282155449637759)","(2.14, 0.16885521842512366)","(-4.08, 2.0526914980023623)","(-9.08, -6.405402795312321)","(7.82, -2.381580092565826)","(5.05, 1.476558463543269)","(4.95, 2.431855955613759)","(-9.17, -5.833015428308029)","(-8.4, -2.287198607538799)","(-8.4, -2.2036381807319567)",...,"(8.59, 0.4023387888190291)","(3.59, 1.2348969089129402)","(-6.84, -4.0807263329938115)","(-9.03, -5.1369561221557145)","(2.82, 0.3130115301111286)","(-1.36, 0.7529049236099103)","(-9.08, -6.072535955246723)","(8.3, 0.8370613586094788)","(5.68, 0.908268072754322)","(-4.81, -3.628734698224675)","(99.0, -3.3234082207916598)","(99.0, 0.7763556554832726)","(99.0, -1.4194395273886393)","(99.0, -4.612044746855682)","(99.0, -2.7950121921583397)","(99.0, 0.889286930516685)","(99.0, -2.1047366038890005)","(-9.42, -0.8278939206105346)","(99.0, -2.7363501731378097)","(99.0, -1.3219726522184407)","(99.0, -0.3182037691094562)","(-7.72, -1.795565225002286)","(99.0, 0.12859233022257782)","(99.0, -2.2553513880715976)","(99.0, -1.791202967948068)","(99.0, -2.293933858025192)","(99.0, 0.010702304857171378)","(99.0, 0.027394316542448116)","(99.0, 2.5338293131532152)","(99.0, -2.6048981484245903)","(2.82, 0.2218502002625136)","(99.0, -1.1264817960062965)","(99.0, 0.5018244355986783)","(99.0, -1.6543442084612139)","(99.0, -1.5897872925414336)","(99.0, -0.8198688954846063)","(99.0, -0.6973163516197783)","(99.0, -1.3704433444163384)","(99.0, -3.3455326748235508)","(99.0, -1.4746468600740903)"
1,100,"(99.0, 1.9074567688158217)","(-0.29, 2.040051954466119)","(6.36, 1.9662253405224905)","(4.37, 1.601455688954893)","(-2.38, 1.602743024199)","(-9.66, 2.6617497084873816)","(-0.73, 1.19367194772322)","(-5.34, 0.6023276797766536)","(8.88, 1.784347481594191)","(9.22, 2.1025858170191127)","(6.75, 2.651593413403034)","(99.0, 2.789731457244113)","(4.42, 0.468256974609062)","(7.43, 2.832071834145356)","(4.56, 0.21087638800774106)","(-0.97, -0.1571081811500652)","(4.66, 0.711682152644162)","(-0.68, 0.9589389189185318)","(3.3, 1.6296690146929658)","(-1.21, 1.3821859186109913)","(0.87, 2.85029624218708)","(8.64, 2.5323201347352917)","(8.35, 2.141158697888758)","(9.17, 1.430877888125288)","(0.05, 2.3971347366160267)","(7.57, 2.728076046459978)","(4.71, 2.7530090177094526)","(0.87, 2.6312824029133526)","(-0.39, 2.555022410897532)","(6.99, 2.2049515629598924)","(6.5, 2.735641055790533)","(-0.92, 2.708173909181642)","(7.14, 1.717191997128369)","(9.03, 2.5236725980481594)","(-1.8, 2.8053985954318077)","(0.73, 2.5780802157602127)","(7.09, 1.3440749251889987)","(3.4, 2.4419964350444308)","(-0.87, 2.659490269163546)",...,"(-6.7, 2.8688859635601816)","(-3.35, 2.8218648501924757)","(-9.03, 2.300365097611679)","(4.47, 1.6809253476045483)","(4.08, 2.742280447167766)","(-3.83, 2.714125101768333)","(8.74, 1.8237500863468101)","(1.12, 2.7009862902230526)","(0.78, 2.763782853529292)","(7.52, 2.391

 Use pandas to find the best- and the worst-rated jokes

In [216]:

total_column=comparison_data.shape[0]
total_row=comparison_data.shape[1]
print(comparison_data.shape)

maxA=-10;
minA=10;
cacheMin=[-1,-1]
cacheMax=[-1,-1]
for y in  range(0,total_column):
  for x in range(3,total_row):  
      x=int(x)
      y=int(y)
      element=comparison_data[x][y]
      #print("{},{},{}".format(y,x,element))
      value1=element[0]
      value2=element[1]
      value=abs(value1-value2)
      if(abs(value1-99.0)>1.0):# dont check prediction 99
        if(value<minA):
          minA=value
          cacheMin=[y,x]
        if(value>maxA):
          maxA=value
          cacheMax=[y,x]
print("best-rated is: {} at location{}, worst rated is: {} at location{}".format(minA,cacheMin,maxA,cacheMax))

(24983, 101)
best-rated is: 6.091625493809261e-06 at location[8228, 29], worst rated is: 19.213792103436532 at location[21578, 54]
